In [1]:
import chess

In [2]:
board = chess.Board()

In [3]:
from board_conversion import *

def generate_side_matrix(board,side):
    matrix = board_matrix(board)
    translate = translate_board(board)
    bools = np.array([piece.isupper() == side for piece in matrix])
    bools = bools.reshape(8,8,1)
    
    side_matrix = translate*bools
    return np.array(side_matrix)

def generate_input(positions,len_positions = 8):
    board_rep = []
    for position in positions:
        black = generate_side_matrix(board,False)
        white = generate_side_matrix(board,True)
        board_rep.append(black)
        board_rep.append(white)
    turn = np.zeros((8,8,12))
    turn.fill(int(board.turn))
    board_rep.append(turn)
    
    while len(board_rep) < len_positions*2 + 1:
        value = np.zeros((8,8,12))
        board_rep.insert(0,value)
    board_rep = np.array(board_rep)
    return board_rep
    
        
input_matrix = generate_input([chess.Board()])

In [4]:
input_matrix.shape

(17, 8, 8, 12)

In [13]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Conv2D, Dense, add, BatchNormalization,Flatten

def residual_module(layer_in, n_filters):
	merge_input = layer_in
	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	batch_norm = BatchNormalization()(conv1)
	layer_out = add([batch_norm, merge_input])
	layer_out = Activation('relu')(layer_out)
	return layer_out

visible = Input(shape=(17, 8, 8,12))
layer1 = residual_module(visible, 64)
flatten = Flatten()(layer1)
p = Dense(4096,activation='softmax',name = 'p')(flatten)
v = Dense(1,activation = 'sigmoid',name = 'v')(flatten)
# create model
model = Model(inputs=visible, outputs=[p,v])
# summarize model
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 17, 8, 8, 12 0                                            
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, 17, 8, 8, 64) 6976        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 17, 8, 8, 64) 256         conv2d_21[0][0]                  
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 17, 8, 8, 64) 832         input_6[0][0]                    
____________________________________________________________________________________________

In [14]:
from keras.losses import CategoricalCrossentropy,BinaryCrossentropy
import keras.optimizers
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)
model.compile(optimizer = optimizer, loss = {'p':CategoricalCrossentropy(),
                                                     'v':BinaryCrossentropy()})

In [15]:
import numpy as np
X = np.random.randn(12,17,8,8,12)
y1 = np.random.randn(12,4096)
y2 = np.random.randn(12,1)
model.fit(X,[y1,y2])

1/1 [==============================] - 6s 6s/step - loss: 379.6279 - p_loss: 377.2117 - v_loss: 2.4162


In [ ]:
from keras.layers import Dense,Flatten
from keras.layers.convolutional import Conv2D
from keras.models import Model, Input
import tensorflow as tf
from board_conversion import *

class Q_model():
    def __init__(self,model = None):
        if model:
            print('CUSTOM MODEL SET')
            self.model = model
        else:
            self.model = self.create_q_model()

    def create_q_model(self):
        def residual_module(layer_in, n_filters):
            merge_input = layer_in
            if layer_in.shape[-1] != n_filters:
                merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
            conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
            batch_norm = BatchNormalization()(conv1)
            layer_out = add([batch_norm, merge_input])
            layer_out = Activation('relu')(layer_out)
            return layer_out

        visible = Input(shape=(17, 8, 8,12))
        layer1 = residual_module(visible, 64)
        layer2 = residual_module(layer1, 64)
        layer3 = residual_module(layer2, 64)
        p = Dense(4096,activation='softmax')(layer3)
        v = Dense(1,activation = 'sigmoid')(layer3)
        return Model(inputs=visible, outputs=[p,v])
    
    def predict(self,env):
        input_values = generate_input(env.position_memory)
        state_tensor = tf.convert_to_tensor(input_values)
        p,v = self.model(state_tensor, training=False)
        return p,v
    
    def explore(self,env):
        action_space = np.random.randn(4096)
        action_space = filter_legal_moves(env.board,action_space)
        action = np.argmax(action_space, axis=None)
        move= num2move[action]
        return move,action
        